In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from prophet import Prophet
from keras.models import load_model, Sequential
from keras.layers import LSTM, Dense, Input
from keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go
from datetime import timedelta
import pickle

# Load the Prophet model
with open('/content/drive/MyDrive/Colab Notebooks/well_production/models/prophet_model.pkl', 'rb') as f:
    prophet_model = pickle.load(f)

# Load the neural network model
nn_model = load_model('/content/drive/MyDrive/Colab Notebooks/well_production/models/ann_model.h5')

# Load the LSTM model
lstm_model = load_model('/content/drive/MyDrive/Colab Notebooks/well_production/models/lstm_model.h5')

# Load the saved scaler
with open('/content/drive/MyDrive/Colab Notebooks/well_production/models/scaler_s.pkl', 'rb') as f:
    scaler = pickle.load(f)

# Define the number of future periods for forecasting
future_periods = 30

# Create future dates for predictions
last_date = pd.to_datetime('2014-01-01')  # Replace with the actual last date of your data
future_dates = pd.date_range(start=last_date, periods=future_periods + 1, freq='D')[1:]

# Prophet predictions
future = pd.DataFrame({'ds': future_dates})
prophet_forecast = prophet_model.predict(future)
prophet_forecast = prophet_forecast[['ds', 'yhat']]
prophet_forecast.set_index('ds', inplace=True)
prophet_forecast.columns = ['Prophet Forecast']

# Handle negative values in Prophet forecast
# prophet_forecast['Prophet Forecast'] = prophet_forecast['Prophet Forecast'].apply(lambda x: max(x, 0))

# Neural network predictions
time_steps = 10  # Adjust based on your model's input requirements

# Create input for forecasting using the last available production data
dummy_data = np.random.rand(time_steps, 1)  # Replace with actual data if available
scaled_data = scaler.transform(dummy_data.reshape(-1, 1)).reshape(1, -1)
nn_forecast = []

for i in range(future_periods):
    predicted_value = nn_model.predict(scaled_data)
    nn_forecast.append(predicted_value[0, 0])
    scaled_data = np.append(scaled_data[:, 1:], predicted_value.reshape(1, 1), axis=1)[:, -time_steps:]

nn_forecast = scaler.inverse_transform(np.array(nn_forecast).reshape(-1, 1))
nn_forecast_df = pd.DataFrame(nn_forecast, index=future_dates, columns=['Neural Network Forecast'])

# LSTM predictions
# Load your actual production data instead of dummy_data
actual_production_data = np.random.rand(100)  # Replace this with actual production data
# Scale the actual production data
scaled_actual_data = scaler.transform(actual_production_data.reshape(-1, 1))

# Prepare the input data for LSTM forecasting
input_data = scaled_actual_data[-time_steps:].reshape(1, time_steps, 1)
lstm_forecast = []

for i in range(future_periods):
    predicted_value = lstm_model.predict(input_data)
    lstm_forecast.append(predicted_value[0, 0])
    input_data = np.append(input_data[:, 1:, :], predicted_value.reshape(1, 1, 1), axis=1)
    input_data = input_data[:, -time_steps:, :]

lstm_forecast = scaler.inverse_transform(np.array(lstm_forecast).reshape(-1, 1))
lstm_forecast_df = pd.DataFrame(lstm_forecast, index=future_dates, columns=['LSTM Forecast'])

# Combine forecasts
combined_forecast = pd.concat([prophet_forecast, nn_forecast_df, lstm_forecast_df], axis=1)

# Plot the combined forecasts
fig = go.Figure()

fig.add_trace(go.Scatter(x=combined_forecast.index, y=combined_forecast['Prophet Forecast'], mode='lines', name='Prophet Forecast'))
fig.add_trace(go.Scatter(x=combined_forecast.index, y=combined_forecast['Neural Network Forecast'], mode='lines', name='Neural Network Forecast'))
fig.add_trace(go.Scatter(x=combined_forecast.index, y=combined_forecast['LSTM Forecast'], mode='lines', name='LSTM Forecast'))

fig.update_layout(title='Oil Production Forecast for the Next 30 Days',
                  xaxis_title='Date',
                  yaxis_title='Production',
                  showlegend=True,
                  template='plotly_dark')

# Show the plot
fig.show()


1/1 [==============================] - 0s 17ms/step


In [ ]:
import pandas as pd
import numpy as np
from prophet import Prophet
from keras.models import load_model, Sequential
from keras.layers import LSTM, Dense, Input, SimpleRNN
from keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go
from datetime import timedelta
import pickle

# Load the Prophet model
with open('/content/drive/MyDrive/Colab Notebooks/well_production/models/prophet_model.pkl', 'rb') as f:
    prophet_model = pickle.load(f)

# Load the neural network model
nn_model = load_model('/content/drive/MyDrive/Colab Notebooks/well_production/models/ann_model.h5')

# Load the LSTM model
lstm_model = load_model('/content/drive/MyDrive/Colab Notebooks/well_production/models/lstm_model.h5')

# Load the RNN model
rnn_model = load_model('/content/drive/MyDrive/Colab Notebooks/well_production/models/rnn_model.h5')

# Load the saved scaler
with open('/content/drive/MyDrive/Colab Notebooks/well_production/models/scaler_s.pkl', 'rb') as f:
    scaler = pickle.load(f)

# Define the number of future periods for forecasting
future_periods = 30

# Create future dates for predictions
last_date = pd.to_datetime('2024-07-21')  # Replace with the actual last date of your data
future_dates = pd.date_range(start=last_date, periods=future_periods + 1, freq='D')[1:]

# Prophet predictions
future = pd.DataFrame({'ds': future_dates})
prophet_forecast = prophet_model.predict(future)
prophet_forecast = prophet_forecast[['ds', 'yhat']]
prophet_forecast.set_index('ds', inplace=True)
prophet_forecast.columns = ['Prophet Forecast']

# Handle negative values in Prophet forecast
prophet_forecast['Prophet Forecast'] = prophet_forecast['Prophet Forecast'].apply(lambda x: max(x, 0))

# Neural network predictions
time_steps = 10  # Adjust based on your model's input requirements

# Create input for forecasting using the last available production data
actual_production_data = np.random.rand(100)  # Replace with actual production data
scaled_actual_data = scaler.transform(actual_production_data.reshape(-1, 1))

# Prepare the input for LSTM
input_data_lstm = scaled_actual_data[-time_steps:].reshape(1, time_steps, 1)
lstm_forecast = []

for i in range(future_periods):
    predicted_value = lstm_model.predict(input_data_lstm)
    lstm_forecast.append(predicted_value[0, 0])
    input_data_lstm = np.append(input_data_lstm[:, 1:, :], predicted_value.reshape(1, 1, 1), axis=1)
    input_data_lstm = input_data_lstm[:, -time_steps:, :]

# Inverse transform the forecasted values
lstm_forecast = scaler.inverse_transform(np.array(lstm_forecast).reshape(-1, 1))
lstm_forecast_df = pd.DataFrame(lstm_forecast, index=future_dates, columns=['LSTM Forecast'])

# Prepare the input for RNN
input_data_rnn = scaled_actual_data[-time_steps:].reshape(1, time_steps, 1)
rnn_forecast = []

for i in range(future_periods):
    predicted_value = rnn_model.predict(input_data_rnn)
    rnn_forecast.append(predicted_value[0, 0])
    input_data_rnn = np.append(input_data_rnn[:, 1:, :], predicted_value.reshape(1, 1, 1), axis=1)
    input_data_rnn = input_data_rnn[:, -time_steps:, :]

# Inverse transform the forecasted values
rnn_forecast = scaler.inverse_transform(np.array(rnn_forecast).reshape(-1, 1))
rnn_forecast_df = pd.DataFrame(rnn_forecast, index=future_dates, columns=['RNN Forecast'])

# Neural network predictions
scaled_data = scaled_actual_data[-time_steps:].reshape(1, -1)
nn_forecast = []

for i in range(future_periods):
    predicted_value = nn_model.predict(scaled_data)
    nn_forecast.append(predicted_value[0, 0])
    scaled_data = np.append(scaled_data[:, 1:], predicted_value.reshape(1, 1), axis=1)[:, -time_steps:]

nn_forecast = scaler.inverse_transform(np.array(nn_forecast).reshape(-1, 1))
nn_forecast_df = pd.DataFrame(nn_forecast, index=future_dates, columns=['Neural Network Forecast'])

# Combine forecasts
combined_forecast = pd.concat([prophet_forecast, nn_forecast_df, lstm_forecast_df, rnn_forecast_df], axis=1)

# Plot the combined forecasts
fig = go.Figure()

fig.add_trace(go.Scatter(x=combined_forecast.index, y=combined_forecast['Prophet Forecast'], mode='lines', name='Prophet Forecast'))
fig.add_trace(go.Scatter(x=combined_forecast.index, y=combined_forecast['Neural Network Forecast'], mode='lines', name='Neural Network Forecast'))
fig.add_trace(go.Scatter(x=combined_forecast.index, y=combined_forecast['LSTM Forecast'], mode='lines', name='LSTM Forecast'))
fig.add_trace(go.Scatter(x=combined_forecast.index, y=combined_forecast['RNN Forecast'], mode='lines', name='RNN Forecast'))

fig.update_layout(title='Oil Production Forecast for the Next 30 Days',
                  xaxis_title='Date',
                  yaxis_title='Production',
                  showlegend=True,
                  template='plotly_dark')

# Show the plot
fig.show()


1/1 [==============================] - 0s 24ms/step
